In [1]:
#Libraries
import requests
from bs4 import BeautifulSoup
import sqlite3
import pandas as pd

In [2]:
#I created a database named Books by using commpand prompt
#Connect to database
connection=sqlite3.connect('Books.db')
cursor=connection.cursor()
#Create table Categories
cursor.execute('''CREATE TABLE Categories(Name TEXT PRIMARY KEY)''')

In [3]:
#Website link
url='http://books.toscrape.com/'
response=requests.request('GET',url)
soup=BeautifulSoup(response.text,'html.parser')

In [4]:
#For loop range equal number of categories
for i in range(1,len(soup.find('ul',attrs={'class':'nav nav-list'}).find_all('li'))):
    Categ_Name=(soup.find('ul',attrs={'class':'nav nav-list'}).find_all('a')[i].get_text().strip())
    #Insert into Categories table
    cursor.execute('''INSERT INTO Categories VALUES(?)''',[Categ_Name])
    #Commit
    connection.commit()
#Close connection
connection.close()

In [5]:
#Categories table
connection=sqlite3.connect('Books.db')
Categories_Table=pd.read_sql('''SELECT * FROM Categories''',connection)
connection.close()
Categories_Table

,Name
0,Travel
1,Mystery
2,Historical Fiction
3,Sequential Art
4,Classics
5,Philosophy
6,Romance
7,Womens Fiction
8,Fiction
9,Childrens


In [6]:
#Create Books table
connection=sqlite3.connect('Books.db')
cursor=connection.cursor()
cursor.execute('''CREATE TABLE Books(Title TEXT,
                                    Category_Name TEXT,
                                    Rate INT,
                                    Price real,
                                    FOREIGN KEY(Category_Name)
                                    REFERENCES Categories(Name))''')

In [7]:
#This function return the category for each book
def get_categ(lnk):
    url=lnk
    response=requests.request('GET',url)
    soup=BeautifulSoup(response.text,'html.parser')
    return soup.find('ul',attrs={'class':'breadcrumb'}).find_all('li')[2].get_text().strip()

In [8]:
#For replacing the string rate with integer rate
rate_dic={'One':1,'Two':2,'Three':3,'Four':4,'Five':5}
#This for loop take range of number of all pages in the website
for i in range(int(soup.find('ul',attrs={'class':'pager'}).find('li').get_text().strip()[10:])):
    #i incremented to be suitable with pages numbers 
    url=f'http://books.toscrape.com/catalogue/page-{i+1}.html'
    #Get data
    response=requests.request('GET',url)
    soup=BeautifulSoup(response.text,'html.parser')
    ordered_lst=soup.find('ol',attrs={'class':'row'})
    #This for loop take range of number of all lists tages in each page
    for j in range(len(ordered_lst.find_all('li'))):
        name=ordered_lst.find_all('li')[j].find('h3').find('a').get('title')
        price=float(ordered_lst.find_all('li')[j].find('p',{'class':'price_color'}).get_text().replace('Â£',''))
        rate=rate_dic[ordered_lst.find_all('li')[j].find('p').get('class')[1]]
        categ_name=get_categ('http://books.toscrape.com/'+'catalogue/'+ordered_lst.find_all('li')[j].find('a').get('href'))
        cursor.execute('''INSERT INTO Books VALUES(?,?,?,?)''',[name,categ_name,rate,price])
        connection.commit()
    print(f"Scraping page number {i+1} done")
connection.close()

Scraping page number 1 done
Scraping page number 2 done
Scraping page number 3 done
Scraping page number 4 done
Scraping page number 5 done
Scraping page number 6 done
Scraping page number 7 done
Scraping page number 8 done
Scraping page number 9 done
Scraping page number 10 done
Scraping page number 11 done
Scraping page number 12 done
Scraping page number 13 done
Scraping page number 14 done
Scraping page number 15 done
Scraping page number 16 done
Scraping page number 17 done
Scraping page number 18 done
Scraping page number 19 done
Scraping page number 20 done
Scraping page number 21 done
Scraping page number 22 done
Scraping page number 23 done
Scraping page number 24 done
Scraping page number 25 done
Scraping page number 26 done
Scraping page number 27 done
Scraping page number 28 done
Scraping page number 29 done
Scraping page number 30 done
Scraping page number 31 done
Scraping page number 32 done
Scraping page number 33 done
Scraping page number 34 done
Scraping page number 35

In [9]:
#Books table
connection=sqlite3.connect('Books.db')
Books_Table=pd.read_sql('''SELECT * FROM Books''',connection)
connection.close()
Books_Table

,Title,Category_Name,Rate,Price
0,A Light in the Attic,Poetry,3,51.77
1,Tipping the Velvet,Historical Fiction,1,53.74
2,Soumission,Fiction,1,50.10
3,Sharp Objects,Mystery,4,47.82
4,Sapiens: A Brief History of Humankind,History,5,54.23
...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,Classics,1,55.53
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",Sequential Art,4,57.06
997,A Spy's Devotion (The Regency Spies of London #1),Historical Fiction,5,16.97
998,1st to Die (Women's Murder Club #1),Mystery,1,53.98
